# Projeto 1 - Ciência dos Dados

Nome: Esther Dagir

Nome: Lila Hadba

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
pd.options.display.max_rows = 13

In [2]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [3]:
import os

filename = 'Netflix.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Netflix.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,"0 irrelevante, 1 relevante, 2 muito relevante"
0,saiu da netflix? q sdd de full house! amo muit...,1
1,rt @karolcaipira: gente vcs tem que entender u...,1
2,rt @yuki_blc: agora que one piece entrará no c...,0
3,"mano, não tem mais nada p assistir na netflix ...",2
4,"@jornaloglobo vcs assistiram o filme, primeira...",0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,0 irrelevante e 1 relevante
0,rt @blogdadireita: min. damares alves quer pro...,0
1,@rachwlgrwn no off eu fui até a terceira e se ...,2
2,eu quero julie e os fantasmas na netflix pra c...,0
3,nunca vou perdoar a netflix https://t.co/ssrbk...,2
4,seria demais querer que a @damaresalves e os e...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é um serviço de streaming de filmes e séries. De uma escala de 0 a 2 temos que:
0 é "nada relevante" - abrange tweets que não influenciam no desempenho da plataforma
1 é "relevante" - tweets que são recomendações ou reinvidicações para a plataforma
2 é "muito relevante" - todos os tweets que estão relacionados ao gosto positivamente e negativamente à plataforma

In [6]:
dfnaorelevante = train.loc[train['0 irrelevante, 1 relevante, 2 muito relevante'] == 0]
dfnaorelevante

,Treinamento,"0 irrelevante, 1 relevante, 2 muito relevante"
2,rt @yuki_blc: agora que one piece entrará no c...,0
4,"@jornaloglobo vcs assistiram o filme, primeira...",0
5,rt @bunnyfeveer: esses dias estreou o filme mi...,0
7,o jeremy shada na seria da netflix e eu so lem...,0
8,damares quer proibir filme da netflix acusado ...,0
...,...,...
290,nada tira da minha cabeça que ela já sabia que...,0
294,os militantes que cancelaram a netflix pelo fi...,0
295,rt @craibeibe: inveja de você? kkkkkkkkkkkkkkk...,0
296,quero saber quando é q vou poder colocar as pi...,0


In [7]:
#juntar num textao
textao0 = " ".join(dfnaorelevante.Treinamento)

In [8]:
dfrelevante = train.loc[train['0 irrelevante, 1 relevante, 2 muito relevante'] == 1]
dfrelevante

,Treinamento,"0 irrelevante, 1 relevante, 2 muito relevante"
0,saiu da netflix? q sdd de full house! amo muit...,1
1,rt @karolcaipira: gente vcs tem que entender u...,1
15,@bancadadeleao tive algumas dificuldades com a...,1
25,@onepiecedadepre finalmente o melhor anime do ...,1
26,"@joaopaulojm16 ""o dilema das redes"" documentár...",1
...,...,...
280,eu só queria minha netflix de volta....😭,1
281,@gaudby saudade de pegar aquela filinha de lev...,1
283,o dilema das redes na netflix é 10/10 e todo m...,1
293,netflix já foi melhor .,1


In [9]:
#juntar num textao
textao1 = " ".join(dfrelevante.Treinamento)

In [10]:
dfmuitorelevante = train.loc[train['0 irrelevante, 1 relevante, 2 muito relevante'] == 2]
dfmuitorelevante

,Treinamento,"0 irrelevante, 1 relevante, 2 muito relevante"
3,"mano, não tem mais nada p assistir na netflix ...",2
6,mano 16ª ja ta no globoplay caraaaaio \nnetfli...,2
13,rt @jovisf30: todos os filmes de harry potter ...,2
16,nesse feriado prolongado eu assisti dois filme...,2
36,rt @wahlianna: netflix be like:\n\ncenas com p...,2
...,...,...
286,@netovisky trilogia de senhor dos anéis (dispo...,2
289,"rt @denise_rm_: bom dia, povo!🌹🇧🇷\nmenos p a n...",2
291,rt @awaecharts: anne with an e na netflix chil...,2
292,rt @baptistafairy: eu tô muito sedenta pela se...,2


In [11]:
#juntar num textao
textao2 = " ".join(dfmuitorelevante.Treinamento)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [12]:
netflix0 = cleanup(textao0.lower())

print(netflix0[:])

rt @yuki_blc agora que one piece entrará no catálogo da netflix brasil torço para que uma nova massa de otakus se dêem a oportunidade de… @jornaloglobo vcs assistiram o filme primeiramente

https//tco/3rio38lney rt @bunnyfeveer esses dias estreou o filme mignonnes ou cuties em inglês na netflix então eu vou fazer minha breve passagem sobre como… o jeremy shada na seria da netflix e eu so lembro da epoca qe ele dublava o lance e fazia a gente acreditar qe klance era canon 👁👄👁 damares quer proibir filme da netflix acusado de sexualizar criança https//tco/nf89cx83hf cultura @elgwellm meu pai tá me pirraçando e até agora não pagou a netflix ainda nem acabei o clássico @pacefeco o que tem a ver a globo com a netflix gente gado não serve nem para criticar kkkkkkkkk me ajudem vi bastante gente falando mas não tive interesse de assistir a netflix lançou uma série soco no estômago expondo como funcionam as redes empresas lucros internet tipo a netflix rt @isalipertin netflix renova caçadoras de

In [13]:
todas_palavras_naorelevantes = netflix0.split()

In [14]:
serie_0 = pd.Series(todas_palavras_naorelevantes)
serie_0

0                          rt
1                   @yuki_blc
2                       agora
3                         que
4                         one
                ...          
4257               importando
4258                        a
4259                     fake
4260                   abaixo
4261    https//tco/tnrpdhzlgj
Length: 4262, dtype: object

In [15]:
tabela_naorelevante = serie_0.value_counts()
tabela_naorelevante

netflix       189
a             132
de            117
que            96
o              91
             ... 
trazer          1
partido         1
programas       1
deitada         1
importando      1
Length: 1614, dtype: int64

In [31]:
#derivacoes_0 = []
#for i in serie_0:
#    if "cancel" in i or "renov" in i:
#        derivacoes_0.append(i)
#print(derivacoes_0)

#damares, pedofilia, infantil, ver, assistir

derivacoes_0 = {}
for i in serie_0:
    if "damares" in i or "pedof" in i or "infant" in i or "ver" in i or "assist" in i:
        if i in derivacoes_0.keys():
            derivacoes_0[i] += 1
        else:
            derivacoes_0[i] = 1
print(derivacoes_0)

{'assistiram': 1, 'damares': 9, 'ver': 20, 'assistir': 15, 'deverá': 1, 'universal': 1, 'assiste': 4, 'assistam': 1, 'never': 46, 'assistido': 2, 'infantil': 2, 'river': 1, 'assisti': 4, 'rever': 1, 'assista': 2, 'assistindo': 6, 'assistidas': 2, 'governo': 1, 'deveriam': 1, 'divertidos': 1, 'tiver': 1, 'assisto': 1, 'pedofilos': 1, '@damaresalves': 6, 'estiver': 1, 'netflixpedofilianão': 1, 'pedofilianao': 1, 'vergonha': 1, '@xadrezverbal': 1, 'divertir': 1}


In [16]:
netflix1 = cleanup(textao1.lower())

print(netflix1[:])

saiu da netflix q sdd de full house amo muitooo https//tco/ux4t89lxb2 rt @karolcaipira gente vcs tem que entender uma coisa nós não sabemos o que realmente aconteceu pra anne ser cancelada 
a cbc não quis… @bancadadeleao tive algumas dificuldades com a opção “forward” encravou imensas vezes a app não tem tanta fluidez como a app da netflix @onepiecedadepre finalmente o melhor anime do mundo aparecendo da netflix carai @joaopaulojm16 o dilema das redes documentário novo da netflix achei bem foda e preocupante ao mesmo tempo essa madrugada assistir a serie julie and the phantoms na netflix é bem clichêzinho adolescente feat high school musical adorei hahahahha rt @murislx netflix pra valorizar o trabalho dos artistas que trabalham anos se dedicando a atuação fazendo cursos faculdade e etc🤬🤬🤬🤬🤬… agiliza né @netflix renewtbh https//tco/x1kz0lpfih netflix que aguente a chuva de “cadê os novos episódios” colocando one piece no catálogo irrita me não ter netflix https//tco/shepthy9gz rt @slon

In [17]:
todas_palavras_relevantes = netflix1.split()

In [18]:
serie_1 = pd.Series(todas_palavras_relevantes)
serie_1

0                       saiu
1                         da
2                    netflix
3                          q
4                        sdd
               ...          
679                     meus
680                lencinhos
681                       de
682                    papel
683    https//tco/edylx1xewt
Length: 684, dtype: object

In [19]:
tabela_relevante = serie_1.value_counts()
tabela_relevante

netflix      38
a            28
que          15
de           13
rt           12
             ..
@netflix      1
madrugada     1
foda          1
globo         1
dorama        1
Length: 390, dtype: int64

In [37]:
#saiu, cancelar, finalmente, renovar, séries

#derivacoes_1 = []
#for i in serie_1:
#    if "cancel" in i or "renov" in i:
#        derivacoes_1.append(i)
#print(derivacoes_1)

derivacoes_1 = {}
for i in serie_1:
    if "saiu" in i or "cancel" in i or "finalmente" in i or "renov" in i or "série" in i:
        if i in derivacoes_1.keys():
            derivacoes_1[i] += 1
        else:
            derivacoes_1[i] = 1
print(derivacoes_1)

{'saiu': 1, 'cancelada': 1, 'finalmente': 1, 'renovou': 1, 'séries': 2, 'série': 2, 'cancelem': 1}


In [20]:
netflix2 = cleanup(textao2.lower())

print(netflix2[:])

mano não tem mais nada p assistir na netflix que goste/queira mano 16ª ja ta no globoplay caraaaaio 
netflix ja é quase seu fim rt @jovisf30 todos os filmes de harry potter no google drive

já que a netflix n posta eu posto 100 favs e posto animais fantásticos e on… nesse feriado prolongado eu assisti dois filmes ruins
 alive e the last days of american crime
as vezes eu me arrependo de pagar caro na netflix rt @wahlianna netflix be like

cenas com personagens fumando 😡😠😡😡🤬😤😤👎🚭🚭❌❌🚫🔞😖😣😱🚫🚫🚩🚩👎😯😮🤬🚭🚭🚭

cenas sexualizando crianças 😂😏😏😍😍😈💗💞👅👍❤️❤️✅😊🤗🤩… eu estava na tour de subir hashtag pra netflix comprar e fazer a segunda temporada de julie e os fantasmas mas néé gostam de pegar as coisas boas de outros países e fazer do jeito deles @damaresalves parabéns ministra damares alves por seu posicionamento contra a pedofilia também peço a vc que notifique a netflix contra o comercial erótico que a empresa está exibindo 
@jairbolsonaro 
@damaresalves 
@amendoncamjsp rt @detailsaluado netflix estrag

In [21]:
todas_palavras_muitorelevantes = netflix2.split()

In [22]:
serie_2 = pd.Series(todas_palavras_muitorelevantes)
serie_2

0           mano
1            não
2            tem
3           mais
4           nada
          ...   
1069          se
1070    consegue
1071      trazer
1072     bakuman
1073        🙏🏽🙏🏽
Length: 1074, dtype: object

In [23]:
tabela_muitorelevante = serie_2.value_counts()
tabela_muitorelevante

netflix    55
a          51
de         25
na         23
e          23
           ..
app         1
with        1
queria      1
feriado     1
valores     1
Length: 534, dtype: int64

In [36]:
#globoplay, sexualizar, crianças, damares, disney, assinatura, greys

#derivacoes_2 = []
#for i in serie_2:
#    if "cancel" in i or "renov" in i:
#        derivacoes_2.append(i)
#print(derivacoes_2)

derivacoes_2 = {}
for i in serie_2:
    if "globoplay" in i or "sexual" in i or "criança" in i or "damares" in i or "disney" in i or "assin" in i or "greys" in i:
        if i in derivacoes_2.keys():
            derivacoes_2[i] += 1
        else:
            derivacoes_2[i] = 1
print(derivacoes_2)

{'globoplay': 3, 'sexualizando': 1, 'crianças': 2, '@damaresalves': 4, 'damares': 1, 'sexualizar': 1, 'assinatura': 3, 'greys': 5, 'disney': 3, '@fanaticsgreys': 1, 'criança': 1}


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [24]:
probabilidade em relevante/ laplace

SyntaxError: invalid syntax (<ipython-input-24-9831e56ee25e>, line 1)

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [ ]:
#http://www.nltk.org/howto/portuguese_en.html 